In [2]:
import pandas as pd
import glob
import os
# read file from Bike share csv
path = r'D:\bootcamp\week_20_Final_Project\BIke sharing\Bike data'                    
all_files = glob.glob(os.path.join(path, "*.csv"))

# read each csv file and concatenate it into one dataframe
df_from_each_file = (pd.read_csv(f,encoding='latin-1') for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [3]:
concatenated_df.head()

,ï»¿Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,Trip Id
0,4581278.0,1547.0,7021,01/01/2019 00:08,Bay St / Albert St,7233,01/01/2019 00:33,King / Cowan Ave - SMART,1296,Annual Member,NaN
1,4581279.0,1112.0,7160,01/01/2019 00:10,King St W / Tecumseth St,7051,01/01/2019 00:29,Wellesley St E / Yonge St (Green P),2947,Annual Member,NaN
2,4581280.0,589.0,7055,01/01/2019 00:15,Jarvis St / Carlton St,7013,01/01/2019 00:25,Scott St / The Esplanade,2293,Annual Member,NaN
3,4581281.0,259.0,7012,01/01/2019 00:16,Elizabeth St / Edward St (Bus Terminal),7235,01/01/2019 00:20,Bay St / College St (West Side) - SMART,283,Annual Member,NaN
4,4581282.0,281.0,7041,01/01/2019 00:19,Edward St / Yonge St,7257,01/01/2019 00:24,Dundas St W / St. Patrick St,1799,Annual Member,NaN


In [4]:
#delete na row

concatenated_df.dropna(subset = ['User Type'], inplace=True)#249 rows

In [7]:
concatenated_df.dtypes

ï»¿Trip Id            float64
Trip  Duration        float64
Start Station Id       object
Start Time             object
Start Station Name     object
End Station Id         object
End Time               object
End Station Name       object
Bike Id                object
User Type              object
Trip Id               float64
dtype: object

In [9]:
#check if there is still na data

concatenated_df.dropna(subset = ['Trip  Duration'], inplace=True)
concatenated_df[concatenated_df['Trip  Duration'].isna()]

,ï»¿Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,Trip Id


In [11]:
# change datetype from datetime to date
concatenated_df['Date']=pd.to_datetime(concatenated_df['Start Time']).dt.date

In [12]:
# count Trips
Trips=concatenated_df.groupby(['Date'])['Bike Id'].count().reset_index(name='counts_trips')

In [13]:
Trips

,Date,counts_trips
0,2019-01-01,1064
1,2019-01-02,2290
2,2019-01-03,2279
3,2019-01-04,3290
4,2019-01-05,2160
...,...,...
1091,2021-12-27,1265
1092,2021-12-28,3054
1093,2021-12-29,3521
1094,2021-12-30,4065


In [14]:
# count unique bikes on trips
Bikes=concatenated_df.groupby(['Date'])['Bike Id'].nunique().reset_index(name='counts_Bike')
Bikes

,Date,counts_Bike
0,2019-01-01,674
1,2019-01-02,1135
2,2019-01-03,955
3,2019-01-04,1381
4,2019-01-05,1053
...,...,...
1091,2021-12-27,756
1092,2021-12-28,1466
1093,2021-12-29,1668
1094,2021-12-30,1785


In [15]:
#count trips by user type= Annual member
Ann_df0 = concatenated_df[concatenated_df['User Type'] == 'Annual Member']
Ann_df=Ann_df0.groupby(['Date'])['Bike Id'].count().reset_index(name='counts_member_annual')
Ann_df

,Date,counts_member_annual
0,2019-01-01,897
1,2019-01-02,2207
2,2019-01-03,2228
3,2019-01-04,3130
4,2019-01-05,1937
...,...,...
1091,2021-12-27,873
1092,2021-12-28,2025
1093,2021-12-29,2440
1094,2021-12-30,2836


In [16]:
#count trips by user type=Casual member
cas_df0 = concatenated_df[concatenated_df['User Type'] == 'Casual Member']
cas_df=cas_df0.groupby(['Date'])['Bike Id'].count().reset_index(name='counts_member_casual')
cas_df

,Date,counts_member_casual
0,2019-01-01,167
1,2019-01-02,83
2,2019-01-03,51
3,2019-01-04,160
4,2019-01-05,223
...,...,...
1090,2021-12-27,392
1091,2021-12-28,1029
1092,2021-12-29,1081
1093,2021-12-30,1229


In [17]:
#Merge all dataframe to one dataframe 
df_tb = pd.merge(Trips,Bikes,on=['Date'])
df_tba= pd.merge(df_tb,Ann_df,on=['Date'])
df_final= pd.merge(df_tba,cas_df,on=['Date'],how='left')

In [18]:
df_final

,Date,counts_trips,counts_Bike,counts_member_annual,counts_member_casual
0,2019-01-01,1064,674,897,167.0
1,2019-01-02,2290,1135,2207,83.0
2,2019-01-03,2279,955,2228,51.0
3,2019-01-04,3290,1381,3130,160.0
4,2019-01-05,2160,1053,1937,223.0
...,...,...,...,...,...
1091,2021-12-27,1265,756,873,392.0
1092,2021-12-28,3054,1466,2025,1029.0
1093,2021-12-29,3521,1668,2440,1081.0
1094,2021-12-30,4065,1785,2836,1229.0


In [19]:
#sum trip duration group by date
td_df=concatenated_df.groupby(['Date'])['Trip  Duration'].sum().reset_index(name='trip_duration')
#merge to final dataframe
df_final2= pd.merge(df_final,td_df,on=['Date'])

In [20]:
#change column order
df_final2=df_final2[['Date','counts_trips','counts_Bike','trip_duration','counts_member_annual','counts_member_casual']]

df_final2

,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual
0,2019-01-01,1064,674,1121463.0,897,167.0
1,2019-01-02,2290,1135,2838818.0,2207,83.0
2,2019-01-03,2279,955,1650338.0,2228,51.0
3,2019-01-04,3290,1381,2269606.0,3130,160.0
4,2019-01-05,2160,1053,1468872.0,1937,223.0
...,...,...,...,...,...,...
1091,2021-12-27,1265,756,955753.0,873,392.0
1092,2021-12-28,3054,1466,2356616.0,2025,1029.0
1093,2021-12-29,3521,1668,2896184.0,2440,1081.0
1094,2021-12-30,4065,1785,2938443.0,2836,1229.0


In [21]:
#save to csv
df_final2.to_csv('result/Bike_data.csv',index=False)

In [22]:
#Read data weather
weather_data_df=pd.read_csv('weather/weather_data_clean.csv')
weather_data_df

,W_date,W_Temp_Max,W_Temp_Avg,W_Temp_Min,W_Max_wind,W_Avg_wind,W_Min_wind,W_Wind_Gust,W_Air_Pressure,W_Snow_Depth,W_Max_humid,W_Avg_humid,W_Min_humid,W_Max_Dp,W_Avg_dp,W_Min_Dp
0,2019-01-01,5.2,-1.4,-8.0,37.0,21.4,3.2,58.0,1015.1,NaN,100.0,85.5,64.0,6.1,-0.6,-11.1
1,2019-01-02,-0.5,-4.9,-9.3,20.9,11.9,8.0,NaN,1023.0,0.0,100.0,84.0,74.0,0.0,-5.1,-11.1
2,2019-01-03,0.8,-1.0,-2.8,37.0,18.2,0.0,46.0,1013.3,30.0,100.0,86.0,69.0,0.0,-1.9,-5.0
3,2019-01-04,8.0,3.6,-0.8,29.0,18.8,8.0,54.0,1007.6,30.0,87.0,80.7,70.0,2.2,0.1,-1.1
4,2019-01-05,4.0,1.2,-1.7,32.2,16.3,3.2,33.0,1006.1,0.0,93.0,83.7,75.0,1.1,-0.1,-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,0.1,-3.1,-6.2,59.5,38.9,9.7,48.0,1014.1,40.0,100.0,90.9,68.0,1.1,-2.3,-10.0
1092,2021-12-28,2.3,0.9,-0.6,40.2,17.2,3.2,57.0,1009.8,20.0,100.0,80.5,56.0,2.2,-1.4,-5.0
1093,2021-12-29,2.2,1.0,-0.3,32.2,20.8,4.8,NaN,1011.7,10.0,100.0,91.1,81.0,2.8,0.9,-1.1
1094,2021-12-30,2.2,1.1,0.0,24.1,12.6,4.8,NaN,1013.8,NaN,87.0,80.5,70.0,1.1,-0.4,-2.2


In [23]:
#Merge data weather and final 
df_final2.Date = pd.to_datetime(df_final2.Date)
weather_data_df.W_date = pd.to_datetime(weather_data_df.W_date)

Merge_data_df=pd.merge(weather_data_df,df_final2,left_on='W_date', right_on='Date',how='left')

In [24]:
#Check data missing for bike data
Merge_data_df[Merge_data_df['Date'].isna()]

,W_date,W_Temp_Max,W_Temp_Avg,W_Temp_Min,W_Max_wind,W_Avg_wind,W_Min_wind,W_Wind_Gust,W_Air_Pressure,W_Snow_Depth,...,W_Min_humid,W_Max_Dp,W_Avg_dp,W_Min_Dp,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual


In [25]:
# Delete row that has no bike data 
#Merge_data_df.drop(Merge_data_df[Merge_data_df['W_date']== '2019-02-12'].index, inplace = True)
Merge_data_df

,W_date,W_Temp_Max,W_Temp_Avg,W_Temp_Min,W_Max_wind,W_Avg_wind,W_Min_wind,W_Wind_Gust,W_Air_Pressure,W_Snow_Depth,...,W_Min_humid,W_Max_Dp,W_Avg_dp,W_Min_Dp,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual
0,2019-01-01,5.2,-1.4,-8.0,37.0,21.4,3.2,58.0,1015.1,NaN,...,64.0,6.1,-0.6,-11.1,2019-01-01,1064,674,1121463.0,897,167.0
1,2019-01-02,-0.5,-4.9,-9.3,20.9,11.9,8.0,NaN,1023.0,0.0,...,74.0,0.0,-5.1,-11.1,2019-01-02,2290,1135,2838818.0,2207,83.0
2,2019-01-03,0.8,-1.0,-2.8,37.0,18.2,0.0,46.0,1013.3,30.0,...,69.0,0.0,-1.9,-5.0,2019-01-03,2279,955,1650338.0,2228,51.0
3,2019-01-04,8.0,3.6,-0.8,29.0,18.8,8.0,54.0,1007.6,30.0,...,70.0,2.2,0.1,-1.1,2019-01-04,3290,1381,2269606.0,3130,160.0
4,2019-01-05,4.0,1.2,-1.7,32.2,16.3,3.2,33.0,1006.1,0.0,...,75.0,1.1,-0.1,-1.1,2019-01-05,2160,1053,1468872.0,1937,223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,0.1,-3.1,-6.2,59.5,38.9,9.7,48.0,1014.1,40.0,...,68.0,1.1,-2.3,-10.0,2021-12-27,1265,756,955753.0,873,392.0
1092,2021-12-28,2.3,0.9,-0.6,40.2,17.2,3.2,57.0,1009.8,20.0,...,56.0,2.2,-1.4,-5.0,2021-12-28,3054,1466,2356616.0,2025,1029.0
1093,2021-12-29,2.2,1.0,-0.3,32.2,20.8,4.8,NaN,1011.7,10.0,...,81.0,2.8,0.9,-1.1,2021-12-29,3521,1668,2896184.0,2440,1081.0
1094,2021-12-30,2.2,1.1,0.0,24.1,12.6,4.8,NaN,1013.8,NaN,...,70.0,1.1,-0.4,-2.2,2021-12-30,4065,1785,2938443.0,2836,1229.0


In [26]:
#Check to make count trips has value
df_final2[df_final2['counts_trips']==0]

,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual


In [27]:
Merge_data_df = Merge_data_df.drop(['W_date'], 1)

C:\Anaconda\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [28]:
#rearrange column 
Merge_data_df = Merge_data_df[['Date','counts_trips','counts_Bike','trip_duration','counts_member_annual','counts_member_casual','W_Temp_Max','W_Temp_Avg','W_Temp_Min','W_Max_wind','W_Avg_wind','W_Min_wind','W_Wind_Gust','W_Air_Pressure','W_Snow_Depth','W_Max_humid','W_Avg_humid','W_Min_humid','W_Max_Dp','W_Avg_dp','W_Min_Dp']]


In [29]:
Merge_data_df

,Date,counts_trips,counts_Bike,trip_duration,counts_member_annual,counts_member_casual,W_Temp_Max,W_Temp_Avg,W_Temp_Min,W_Max_wind,...,W_Min_wind,W_Wind_Gust,W_Air_Pressure,W_Snow_Depth,W_Max_humid,W_Avg_humid,W_Min_humid,W_Max_Dp,W_Avg_dp,W_Min_Dp
0,2019-01-01,1064,674,1121463.0,897,167.0,5.2,-1.4,-8.0,37.0,...,3.2,58.0,1015.1,NaN,100.0,85.5,64.0,6.1,-0.6,-11.1
1,2019-01-02,2290,1135,2838818.0,2207,83.0,-0.5,-4.9,-9.3,20.9,...,8.0,NaN,1023.0,0.0,100.0,84.0,74.0,0.0,-5.1,-11.1
2,2019-01-03,2279,955,1650338.0,2228,51.0,0.8,-1.0,-2.8,37.0,...,0.0,46.0,1013.3,30.0,100.0,86.0,69.0,0.0,-1.9,-5.0
3,2019-01-04,3290,1381,2269606.0,3130,160.0,8.0,3.6,-0.8,29.0,...,8.0,54.0,1007.6,30.0,87.0,80.7,70.0,2.2,0.1,-1.1
4,2019-01-05,2160,1053,1468872.0,1937,223.0,4.0,1.2,-1.7,32.2,...,3.2,33.0,1006.1,0.0,93.0,83.7,75.0,1.1,-0.1,-1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2021-12-27,1265,756,955753.0,873,392.0,0.1,-3.1,-6.2,59.5,...,9.7,48.0,1014.1,40.0,100.0,90.9,68.0,1.1,-2.3,-10.0
1092,2021-12-28,3054,1466,2356616.0,2025,1029.0,2.3,0.9,-0.6,40.2,...,3.2,57.0,1009.8,20.0,100.0,80.5,56.0,2.2,-1.4,-5.0
1093,2021-12-29,3521,1668,2896184.0,2440,1081.0,2.2,1.0,-0.3,32.2,...,4.8,NaN,1011.7,10.0,100.0,91.1,81.0,2.8,0.9,-1.1
1094,2021-12-30,4065,1785,2938443.0,2836,1229.0,2.2,1.1,0.0,24.1,...,4.8,NaN,1013.8,NaN,87.0,80.5,70.0,1.1,-0.4,-2.2


In [32]:
#fill NaN value to 0 
Merge_data_df=Merge_data_df.fillna(0)

In [34]:
#save to csv
Merge_data_df.to_csv('result/Bike_weather_merge.csv',index=False)